# Measure Impact using Semantic Scholar

### Neil D. Lawrence 7th June 2021

This notebook looks at the actual impact of the papers published using the Semantic Scholar data base for tracking citations.

In [ ]:
%pip install semanticscholar

In [ ]:
import semanticscholar as sch

In [ ]:
import cmtutils as cu

In [ ]:
papers = cu.Papers()

https://proceedings.neurips.cc/paper/2014 

In [ ]:
import pandas as pd

In [ ]:

id = pd.read_excel('', 
                   sheet_name="Semantic Impact", 
                   converters={"ID" : int})
id = id.set_index(id['ID'].apply(str))

In [ ]:
id['S2CRID']

In [ ]:
import numpy as np
paper_semantic = {}
for index, s2crid in id['S2CRID'].items():
    if s2crid != "?" and not np.isnan(s2crid):
        paper_semantic[str(index)] = str(s2crid)

In [ ]:
len(paper_semantic.keys())

In [ ]:
key = '1788'

In [ ]:
key not in sscholar or sscholar[key] == {}


In [ ]:
sscholar[key] = sch.paper('CorpusID:'+paper_semantic[key], timeout=10)

In [ ]:
from requests.exceptions import ReadTimeout
#sscholar = {}

In [ ]:
for key in paper_semantic:
    if key not in sscholar or sscholar[key] == {}:
        try:
            sscholar[key] = sch.paper('CorpusID:'+paper_semantic[key], timeout=10)
        except ReadTimeout as e:
            key_missing.append(key)
            print("Read time out for key ", key)

In [ ]:
key

In [ ]:
sscholar[key]

In [ ]:
ssdf = pd.DataFrame.from_dict(sscholar, orient="index")

In [ ]:
sscholar['1788'] == {}

In [ ]:
decisions = pd.read_excel('', sheet_name="Final Decisions", converters={"index" : int})
decisions = decisions.set_index(decisions['index'].apply(str))

In [ ]:
def avg_vals(vals):
    if type(vals) is not str:
        return np.NaN
     
    lvals = vals.split(",")
    return np.mean([float(val) for val in lvals])

In [ ]:
col_list = ['quality', 'impact', 'calibrated_quality', 'confidence']
for col in col_list:
    decisions['average_' + col] = decisions[col].apply(avg_vals)

In [ ]:
joindf = decisions.join(ssdf)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(joindf['average_impact'], np.log10(1+ joindf['numCitedBy']), 'r.')

In [ ]:
plt.plot(joindf['average_calibrated_quality'], np.log10(1+ joindf['numCitedBy']), 'r.')

In [ ]:
plt.plot(joindf['average_calibrated_quality'], joindf['numCitedBy'], 'r.')

In [ ]:
joindf['average_impact'].corr(np.log(1+joindf['numCitedBy']))

In [ ]:
joindf['average_calibrated_quality'].corr(np.log(1+joindf['numCitedBy']))

In [ ]:
joindf['average_quality'].corr(np.log(1+joindf['numCitedBy']))

In [ ]:
joindf['average_confidence'].corr(np.log(1+joindf['numCitedBy']))

In [ ]:
plt.plot(joindf['average_confidence'], np.log(1+joindf['numCitedBy']), 'r.')

In [ ]:
np.log10(1+ joindf['numCitedBy']).hist(bins=100)

In [ ]:
joindf['numCitedBy']['1313']

In [ ]:
joindf['numCitedBy'].isna()

In [ ]:
papers.papers.loc['423']

In [ ]:
import googoal